In [ ]:
import polars as pl
import plotly
import plotly.express as px
import pandas as pd
# 1. Definir las rutas de los archivos
csv_path = r"C:\Users\danti\Downloads\SNIBEjemplares_20250710_004212\SNIBEjemplares_20250710_004212.csv"
parquet_path = r"C:\Users\danti\Downloads\SNIBEjemplares_20250710_004212\SNIBEjemplares.parquet"
df_lazy = pl.scan_csv(csv_path)
df_lazy.sink_parquet(parquet_path)
snib_lazy_df = pl.scan_parquet(parquet_path)
snib_lazy_df.collect_schema()
# =====================================
# 1. Configuración base  otras
# =====================================
valores_excluir = ["NO APLICA", " ", "NO DISPONIBLE", "", "NaN", None]
N = 10
# =====================================
# 2. Calcular Top 10 y "Otras"
# =====================================
# Conteo total por año y colección
conteos = (
    snib_lazy_df
    .filter(
        pl.col("aniocolecta").is_not_null() &
        pl.col("coleccion").is_not_null() &
        ~pl.col("coleccion").is_in(valores_excluir)
    )
    .with_columns(
        pl.when(pl.col("aniocolecta").cast(pl.Utf8).str.strip_chars().is_in(["null", "", "0", " "]))
        .then(None)
        .otherwise(pl.col("aniocolecta"))
        .alias("aniocolecta")
    )
    .with_columns(pl.col("aniocolecta").cast(pl.Int32))
    .filter(pl.col("aniocolecta").is_not_null())
    .group_by(["aniocolecta", "coleccion"])
    .agg(pl.len().alias("conteo"))

)
# Asignar ranking por año
ranked = conteos.with_columns(
    pl.col("conteo")
    .rank(method="dense", descending=True)
    .over("aniocolecta")
    .alias("rank")
)
# --- Parte A: Top 10 ---
top_n = ranked.filter(pl.col("rank") <= N)

# --- Parte B: "Otras" ---
otras = (
    ranked.filter(pl.col("rank") > N)
    .group_by("aniocolecta")
    .agg(pl.col("conteo").sum().alias("conteo"))
    .with_columns(pl.lit("Otras").alias("coleccion"))

)
# --- Unir Top 10 + Otras ---
# Aseguramos que ambas tablas tengan las mismas columnas
top_n_clean = top_n.select(["aniocolecta", "coleccion", "conteo"])
otras_clean = otras.select(["aniocolecta", "coleccion", "conteo"])

final_df = (
    pl.concat([top_n_clean, otras_clean])
    .sort(["aniocolecta", "conteo"], descending=[False, True])
    .collect()
)

# =====================================
# 3. Orden para gráfico
# =====================================
years_in_descending_order = sorted(final_df["aniocolecta"].drop_nulls().unique().to_list(), reverse=True)
collections_in_order = final_df["coleccion"].unique().to_list()
# =====================================
# 4. Gráfico Plotly
# =====================================
fig = px.bar(
    final_df,
    x="aniocolecta",
    y="conteo",
    color="coleccion",
    title=f'Top {N} Colecciones por Año - SNIB (con grupo "Otras")',
    labels={
        "aniocolecta": "Año de Colecta",
        "conteo": "Número Total de Registros",
        "coleccion": "Colección"
    },
    category_orders={
        "aniocolecta": years_in_descending_order,
        "coleccion": collections_in_order
    },
    color_discrete_sequence=px.colors.qualitative.Alphabet,
    template="plotly_white"
)

fig.show()
